In [2]:
from pathlib import Path
from src.data import BNDataset
from src.models.BNNet import BNNet
from src.train import train
from src.constants import HEPAR_TARGET, ALARM_TARGET, GLOBAL_SEED
from src.utils import (get_train_test_splits, encode_data,
                       construct_adj_mat, create_random_dag,
                       update_bn_model, get_terminal_connection_nodes)
from src.models.BNNet import BNNet
import torch

from pgmpy.utils import get_example_model
from pgmpy.metrics import structure_score
from torch.utils.data import DataLoader

import yaml

In [3]:
bn_name = 'alarm'
fpath_config = "/home/gaurang/bayesian_network/code/src/config.yaml"
with open(fpath_config, "r") as f:
    config = yaml.safe_load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
bn = get_example_model("hepar2")
df_data = bn.simulate(10000, seed=GLOBAL_SEED)

  0%|          | 0/70 [00:00<?, ?it/s]

/home/gaurang/anaconda3/envs/gnn/lib/python3.10/site-packages/pgmpy/utils/mathext.py:83: UserWarning: Probability values don't exactly sum to 1. Differ by: -9.99999993922529e-09. Adjusting values.
  warn(


In [5]:
_, _, df_data_test_BIF = get_train_test_splits(df_data, GLOBAL_SEED, overfit=True)
gt_BIF = structure_score(bn, df_data_test_BIF)
gt_BIF

-219.10621316519894

In [6]:
# encode data
df_data, encoder = encode_data(df_data, bn)
df_train, df_valid, df_test = get_train_test_splits(df_data, GLOBAL_SEED, overfit=True)

In [7]:
rand_edge_probability = 0.2
target_node = ALARM_TARGET

In [8]:
adj_df_gt = construct_adj_mat(edge_list=list(bn.edges), nodes=list(bn.nodes), target_node=target_node)
adj_df = create_random_dag(
            nodes=list(bn.nodes),
            target_node=target_node,
            prob=rand_edge_probability,
            num_edges_gt=len(bn.edges),
        )
bn = update_bn_model(bn, adj_df)

In [9]:
train_set = BNDataset(
        df_data=df_train,
        target_node=target_node,
        bn=bn,
        adj_df=adj_df,
        rand_edge_probability=rand_edge_probability,
    )
val_set = BNDataset(
    df_data=df_valid,
    target_node=target_node,
    bn=bn,
    adj_df=adj_df,
    rand_edge_probability=rand_edge_probability,
)
test_set = BNDataset(
    df_data=df_test,
    target_node=target_node,
    bn=bn,
    adj_df=adj_df,
    rand_edge_probability=rand_edge_probability,
)

In [10]:
dataloader_train = DataLoader(train_set, batch_size=config["batch_size_train"])
dataloader_valid = DataLoader(val_set, batch_size=config["batch_size_val"])
dataloader_test = DataLoader(test_set, batch_size=config["batch_size_test"])

In [11]:
edge_index = train_set.edge_index
model = BNNet(
    config=config,
    num_nodes=len(train_set.input_nodes),
    node_states=train_set.input_states,
    edge_index=edge_index,
    terminal_node_ids=train_set.terminal_node_ids,
    target_node_states=train_set.target_states,
)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])

In [14]:
model.MLP

Sequential(
  (0): Linear(in_features=144, out_features=16, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=16, out_features=3, bias=True)
)

In [15]:
batch = next(iter(dataloader_train))

In [16]:
X, y = batch
X = 
print(X.shape, y.shape)

torch.Size([24, 36]) torch.Size([24])


In [17]:
pred = model(X)

In [18]:
criterion(pred, y)

tensor(1.1579, grad_fn=<NllLossBackward0>)